## 필요한 Module DownLoad

In [1]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


## 필요한 Library Import

In [2]:
from scipy.spatial import distance as dist

In [3]:
from imutils import perspective

In [4]:
from imutils import contours

In [5]:
import numpy as np

In [6]:
import argparse

In [7]:
import imutils

In [8]:
import cv2

## midpoint 함수 생성

In [9]:
def midpoint(ptA,ptB):
    return ((ptA[0]+ptB[0])*0.5,(ptA[1]+ptB[1])*0.5)

## Construct the argument parse and parse the arguments

In [10]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,help="path to the input image")
ap.add_argument("-w", "--width", type=float, required=True,help="width of the left-most object in the image (in inches)")
args = vars(ap.parse_args)

## Image Load

In [11]:
image = cv2.imread("./img/sample01.jpg")
image = cv2.resize(image, None, fx=3, fy=3)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)

## 가장자리 찾기

In [12]:
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)

## find contours in the edge map

In [13]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [14]:
cnts

(array([[[343, 611]],
 
        [[343, 614]],
 
        [[344, 614]],
 
        [[343, 613]]], dtype=int32),
 array([[[301, 610]],
 
        [[300, 611]],
 
        [[300, 614]],
 
        [[300, 612]],
 
        [[301, 611]]], dtype=int32),
 array([[[434, 609]],
 
        [[433, 610]],
 
        [[432, 610]],
 
        [[431, 611]],
 
        [[430, 611]],
 
        [[429, 612]],
 
        [[427, 612]],
 
        [[426, 611]],
 
        [[417, 611]],
 
        [[416, 612]],
 
        [[409, 612]],
 
        [[408, 613]],
 
        [[408, 614]],
 
        [[409, 614]],
 
        [[409, 613]],
 
        [[410, 612]],
 
        [[416, 612]],
 
        [[417, 611]],
 
        [[426, 611]],
 
        [[427, 612]],
 
        [[429, 612]],
 
        [[430, 611]],
 
        [[432, 611]],
 
        [[434, 609]],
 
        [[445, 609]],
 
        [[446, 610]],
 
        [[447, 610]],
 
        [[446, 609]]], dtype=int32),
 array([[[391, 606]],
 
        [[391, 608]],
 
        [[390, 609]],
 
 

In [15]:
(cnts, _) = contours.sort_contours(cnts)
pixelsPerMetric = None

In [16]:
# loop over the contours individually
for c in cnts:
    # if the contour is not sufficiently large, ignore it
    if cv2.contourArea(c) < 100:
        continue
    # compute the rotated bounding box of the contour
    orig = image.copy()
    box = cv2.minAreaRect(c)
    print(box)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    print(box)
    box = np.array(box, dtype="int")
    print(box)
    # order the points in the contour such that they appear
    # in top-left, top-right, bottom-right, and bottom-left
    # order, then draw the outline of the rotated bounding
    # box
    box = perspective.order_points(box)
    print(box.dtype)
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
    print(box)
    # loop over the original points and draw them
    for (x, y) in box:
        cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
    # unpack the ordered bounding box, then compute the midpoint
    # between the top-left and top-right coordinates, followed by
    # the midpoint between bottom-left and bottom-right coordinates
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)
    # compute the midpoint between the top-left and top-right points,
    # followed by the midpoint between the top-righ and bottom-right
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)
    # draw the midpoints on the image
    cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
    # draw lines between the midpoints
    cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),(255, 0, 255), 2)
    cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),(255, 0, 255), 2)
    # compute the Euclidean distance between the midpoints
    dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
    dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
    # if the pixels per metric has not been initialized, then
    # compute it as the ratio of pixels to supplied metric
    # (in this case, inches)
    if pixelsPerMetric is None:
        pixelsPerMetric = dB / 3
    # compute the size of the object
    dimA = dA / pixelsPerMetric
    dimB = dB / pixelsPerMetric
    # draw the object sizes on the image
    cv2.putText(orig, "{:.1f}in".format(dimA),(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (0, 0, 0), 2)
    cv2.putText(orig, "{:.1f}in".format(dimB),(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (0, 0, 0), 2)
    # show the output image
    cv2.imshow("Image", orig)
    cv2.waitKey(0)

((9.75000286102295, 578.7500610351562), (55.86143493652344, 46.66904067993164), 45.0)
[[-26.499996 575.50006 ]
 [  6.5      542.50006 ]
 [ 46.       582.00006 ]
 [ 13.000006 615.00006 ]]
[[-26 575]
 [  6 542]
 [ 46 582]
 [ 13 615]]
float32
[[  6. 542.]
 [ 46. 582.]
 [ 13. 615.]
 [-26. 575.]]
((22.47081756591797, 531.5385131835938), (87.19705200195312, 42.22172927856445), 37.14668655395508)
[[-2.5029182e+01  5.2203851e+02]
 [ 4.6673584e-01  4.8838388e+02]
 [ 6.9970818e+01  5.4103851e+02]
 [ 4.4474899e+01  5.7469312e+02]]
[[-25 522]
 [  0 488]
 [ 69 541]
 [ 44 574]]
float32
[[  0. 488.]
 [ 69. 541.]
 [ 44. 574.]
 [-25. 522.]]
((11.0, 384.5), (25.0, 22.0), 90.0)
[[-7.654043e-16  3.720000e+02]
 [ 2.200000e+01  3.720000e+02]
 [ 2.200000e+01  3.970000e+02]
 [ 0.000000e+00  3.970000e+02]]
[[  0 372]
 [ 22 372]
 [ 22 397]
 [  0 397]]
float32
[[  0. 372.]
 [ 22. 372.]
 [ 22. 397.]
 [  0. 397.]]


KeyboardInterrupt: 

In [ ]:
$ python Estimaitng_Length.ipykernel --image ./img/sample01.jpg --width 0.955